In [1]:
import geopandas as gp
from shapely import Point, LineString
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
raw_full_trip_gdf = gp.read_file("../data/freemove/freemove_raw.geojson", geometry='geometry')

In [3]:
# get start and end points

# Create SP and EP columns
raw_full_trip_gdf['TRIP_SP'] = raw_full_trip_gdf.geometry.apply(lambda x: Point(x.coords[0]))
raw_full_trip_gdf['TRIP_EP'] = raw_full_trip_gdf.geometry.apply(lambda x: Point(x.coords[-1]))

# Rename columns and drop unnecessary columns
raw_full_trip_gdf.rename(columns={
    'start_t': 'TRIP_START', 
    'end_t': 'TRIP_END', 
    'traj_id': 'TRIP_ID',
    'length': 'TRIP_LEN_IN_MTRS',
    'user_id': 'PERSON_ID'}, inplace=True)
    
raw_full_trip_gdf.drop(columns=['direction'], axis=1, inplace=True)

In [4]:
EPZ_RADIUS = 200

raw_full_trip_gdf['START_EPZ'] = raw_full_trip_gdf.TRIP_SP.apply(lambda x: x.buffer(EPZ_RADIUS))
raw_full_trip_gdf['END_EPZ'] = raw_full_trip_gdf.TRIP_EP.apply(lambda x: x.buffer(EPZ_RADIUS))

In [5]:
# check whether radius is correct (2*r)
from scipy.spatial import distance

distance.euclidean(raw_full_trip_gdf.START_EPZ[0].exterior.coords[0], raw_full_trip_gdf.START_EPZ[0].exterior.coords[32])


400.0

In [6]:
def cut_epz(row):
    try:
        return LineString([x for x in row.geometry.coords if not Point(x).within(row.START_EPZ) and not Point(x).within(row.END_EPZ)])
    except Exception as e:
        return LineString()

In [7]:
raw_full_trip_gdf['NR_POINTS'] = raw_full_trip_gdf.geometry.apply(lambda x: len(x.coords))
# raw_full_trip_gdf = raw_full_trip_gdf.query('NR_POINTS > 10')

In [9]:
raw_full_trip_gdf['geometry_private'] = raw_full_trip_gdf.progress_apply(lambda x: cut_epz(x), axis = 1)

  0%|          | 0/1408 [00:00<?, ?it/s]

In [21]:
m = gp.GeoDataFrame(raw_full_trip_gdf, geometry='START_EPZ')[['START_EPZ']].head(2).explore()

gp.GeoDataFrame(raw_full_trip_gdf, geometry='geometry_private')[['geometry_private']].head(2).explore(m=m)

TypeError: 'NoneType' object is not subscriptable

In [25]:
gp.GeoDataFrame(raw_full_trip_gdf, geometry='geometry_private').head(2).explore()

TypeError: 'NoneType' object is not subscriptable

In [28]:
gp.GeoDataFrame(raw_full_trip_gdf, geometry='geometry_private').head(2).explore()

TypeError: 'NoneType' object is not subscriptable